Importing libraries

In [23]:
import numpy as np
import tensorflow as  tf
import random

 BC   u(t,1) = u(t,-1) = 0

Collocation point creation

In [24]:
n_c = 5000  #collocation points

def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.uniform(start, end)) 
  
    return res

x_c = Rand(-1, 1, n_c)
t_c = Rand(0, 1, n_c)
x_c = tf.cast(x_c ,dtype='float64')
t_c = tf.cast(t_c ,dtype='float64')
input_c = tf.stack((x_c, t_c), axis=1)
x_bc_u = np.ones((n_c,), dtype='float64')
x_bc_b = - x_bc_u
x_bc_u = tf.cast(x_bc_u, dtype='float64')
x_bc_b = tf.cast(x_bc_b, dtype='float64')
input_bcu = tf.stack((x_bc_u, t_c), axis = 1)
input_bcb = tf.stack((x_bc_b, t_c), axis = 1)
input_bc = tf.concat((input_bcu, input_bcb), axis=0)
input_f = tf.concat((input_c, input_bc), axis=0)
y_bc = 0
out_bc = (y_bc* np.ones((2*n_c,), dtype='float64'))
out_bc = tf.cast(out_bc, dtype='float64')

In [ ]:
x_bc_u = np.ones((n_c,), dtype='float64')
x_bc_b = - x_bc_u
input_bcu = tf.stack((x_bc_u, t_c), axis = 1)
input_bcb = tf.stack((x_bc_b, t_c), axis = 1)
input_bc = np.vstack((input_bcu, input_bcb))
input_bc = tf.cast(input_bc, dtype='float64')
xt_in = np.vstack((input_c, input_bc))
xt_in = tf.cast(xt_in, dtype='float64')
y_bc = 0.0
global u_bc
u_bc = (y_bc* np.ones((2*n_c,), dtype='float64')).T
u_bc = tf.cast(u_bc, dtype='float64')

In [22]:
print(input_bc)

tf.Tensor(
[[ 1.          0.19892341]
 [ 1.          0.95686156]
 [ 1.          0.15390612]
 ...
 [-1.          0.85796386]
 [-1.          0.16361488]
 [-1.          0.65810686]], shape=(10000, 2), dtype=float64)


Architecture ANN

In [25]:
pinn1 = tf.keras.models.Sequential()
pinn1.add(tf.keras.layers.Dense(2,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))
pinn1.add(tf.keras.layers.Dense(50,activation='tanh'))

pinn1.add(tf.keras.layers.Dense(1,activation='tanh'))         #output layer

Custom Loss function

In [26]:
def custom_loss_1(arch, xt):
  x= xt[:, 0:1]
  t= xt[:, 1:2]
  def loss(u_train, u_pred):
    with tf.GradientTape() as t1:
      t1.watch(x)
      t1.watch(t)
      with tf.GradientTape() as t2:
        t2.watch(x)
        xt = tf.stack((x, t), axis=1)
        u = arch(xt)
        u_x = t2.gradient(u, x)
        del t2
      u_t = t1.gradient(u, t)
      u_xx = t1.gradient(u_x, x)
      del t1
    u_pred = tf.cast(u_pred, dtype='float64')
    u_train = tf.cast(u_train, dtype='float64')      
    f = u_t + u*u_x - (0.01/tf.pi)*u_xx
    loss_res = tf.reduce_mean(tf.square(f))
    loss_2 = tf.reduce_mean(tf.square(u_pred-u_bc))
    return loss_res + loss_2
  return loss


loss_fn = custom_loss_1(pinn1, input_f)
pinn1.compile(optimizer='adam', loss=loss_fn, metrics=['mse'])
pinn1.fit(input_bc, out_bc, batch_size=32, epochs=1000)



Epoch 1/1000


ValueError: ignored

Test case

In [ ]:
x_test = np.array(random.uniform(-1, 1), dtype='float64')
t_test = np.array(random.uniform(0, 1), dtype='float64')
xt_test = tf.stack((x_test, t_test), axis=1)
u = pinn1(xt_test)